<a href="https://colab.research.google.com/github/OsvaldoUfla/Jogo-dos-Oito-Metodos-de-Busca-/blob/main/Jogo_dos_Oito_Metodos_de_Busca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Jogo dos Oito (Métodos de Busca) ##

Alunos:
* Leonardo Gonçalves Flora
* Osvaldo Rodrigues de Faria Júnior


In [2]:
import numpy as np
from queue import Queue, PriorityQueue
from ipywidgets import widgets, HBox, VBox
from IPython.display import display, clear_output
import time

# estado final desejado do quebra cabeça
ESTADO_FINAL = np.array([[1, 2, 3], [8, 0, 4], [7, 6, 5]])

def encontrar_zero(estado):
    """Encontra a posição do zero (espaço vazio) no tabuleiro."""
    return tuple(np.argwhere(estado == 0)[0])  # retorna a posição do zero como uma tupla

def movimentos_validos(estado):
    """Retorna uma lista de movimentos válidos para o estado atual."""
    movimentos = []
    posicao_zero = encontrar_zero(estado)  # encontra a posição do zero no estado atual
    if posicao_zero[0] > 0:
        movimentos.append('acima')
    if posicao_zero[0] < 2:
        movimentos.append('abaixo')
    if posicao_zero[1] > 0:
        movimentos.append('esquerda')
    if posicao_zero[1] < 2:
        movimentos.append('direita')
    return movimentos  # retorna lista de movimentos válidos

def mover(estado, direcao):
    """Executa um movimento no tabuleiro e retorna o novo estado, ou retorna None se o movimento for inválido."""
    posicao_zero = encontrar_zero(estado)  # encontra a posição do zero no estado atual
    novo_estado = estado.copy()  # copia o estado para evitar alterações indesejadas
    if direcao == 'acima':
        nova_posicao = (posicao_zero[0] - 1, posicao_zero[1])  # calcula a nova posição se mover para cima
    elif direcao == 'abaixo':
        nova_posicao = (posicao_zero[0] + 1, posicao_zero[1])  # calcula a nova posição se mover para baixo
    elif direcao == 'esquerda':
        nova_posicao = (posicao_zero[0], posicao_zero[1] - 1)  # calcula a nova posição se mover para esquerda
    elif direcao == 'direita':
        nova_posicao = (posicao_zero[0], posicao_zero[1] + 1)  # calcula a nova posição se mover para direita
    else:
        return None  # retorna none se a direção não for reconhecida

    if nova_posicao[0] < 0 or nova_posicao[0] > 2 or nova_posicao[1] < 0 or nova_posicao[1] > 2:
        return None  # retorna none se a nova posição estiver fora dos limites do tabuleiro

    novo_estado[posicao_zero], novo_estado[nova_posicao] = novo_estado[nova_posicao], novo_estado[posicao_zero]
    return novo_estado  # retorna o novo estado após o movimento

def bfs(estado_inicial):
    """Executa a busca em largura para encontrar a solução."""
    visitados = set()  # conjunto para armazenar estados visitados
    fila = Queue()  # fila para a busca em largura
    fila.put((estado_inicial, []))  # adiciona o estado inicial com um caminho vazio à fila

    while not fila.empty():
        estado_atual, caminho = fila.get()  # obtém o próximo estado e caminho da fila
        visitados.add(estado_atual.tobytes())  # adiciona o estado atual convertido em bytes aos visitados

        if np.array_equal(estado_atual, ESTADO_FINAL):  # verifica se o estado atual é o estado final desejado
            return caminho  # retorna o caminho até o estado final

        for direcao_movimento in movimentos_validos(estado_atual):  # para cada movimento válido
            novo_estado = mover(estado_atual, direcao_movimento)  # move para o novo estado
            if novo_estado.tobytes() not in visitados:  # se o novo estado não foi visitado
                fila.put((novo_estado, caminho + [direcao_movimento]))  # adiciona à fila com o novo caminho

    return None

def heuristica(state):
    """Calcula a heurística (distância de Manhattan) para o algoritmo A*."""
    distancia = 0
    for x in range(3):
        for y in range(3):
            valor = state[x, y]  # obtém o valor atual no estado
            if valor != 0:  # calcula a distância de Manhattan se não for o espaço vazio
                objetivo_x, objetivo_y = divmod(valor - 1, 3)  # calcula a posição objetivo do valor
                distancia += abs(objetivo_x - x) + abs(objetivo_y - y)  # soma a distância de Manhattan
    return distancia

def a_star(estado_inicial):
    """Executa o algoritmo A* para encontrar a solução."""
    visitados = set()  # conjunto para armazenar estados visitados
    fila_prioridade = PriorityQueue()  # fila de prioridade para o A*
    fila_prioridade.put((0, estado_inicial.tobytes(), []))  # adiciona o estado inicial com custo 0 e caminho vazio

    while not fila_prioridade.empty():
        _, estado_atual_bytes, caminho = fila_prioridade.get()  # obtém o próximo estado e caminho da fila de prioridade
        estado_atual = np.frombuffer(estado_atual_bytes, dtype=estado_inicial.dtype).reshape(estado_inicial.shape)
        visitados.add(estado_atual_bytes)  # adiciona o estado atual convertido em bytes aos visitados

        if np.array_equal(estado_atual, ESTADO_FINAL):  # verifica se o estado atual é o estado final desejado
            return caminho  # retorna o caminho até o estado final

        for direcao_movimento in movimentos_validos(estado_atual):  # para cada movimento válido
            novo_estado = mover(estado_atual, direcao_movimento)  # move para o novo estado
            novo_estado_bytes = novo_estado.tobytes()
            if novo_estado_bytes not in visitados:  # se o novo estado não foi visitado
                novo_caminho = caminho + [direcao_movimento]  # atualiza o caminho com o novo movimento
                custo = len(novo_caminho) + heuristica(novo_estado)  # calcula o custo do movimento
                fila_prioridade.put((custo, novo_estado_bytes, novo_caminho))  # adiciona à fila de prioridade

    return None

def exibir_tabuleiro(estado):
    """Exibe o tabuleiro na interface."""
    for i in range(3):
        for j in range(3):
            botoes[i][j].description = str(estado[i, j]) if estado[i, j] != 0 else ' '  # atualiza o texto nos botões
            if estado[i, j] == 0:
                botoes[i][j].style.button_color = 'lightgray'
                botoes[i][j].style.font_color = 'black'
            else:
                botoes[i][j].style.button_color = 'blue'
                botoes[i][j].style.font_color = 'white'

def on_click_movimento(botao):
    """Callback para os botões de movimento."""
    global estado_atual  # usa a variável global estado_atual
    direcao_movimento = botao.description.lower()  # obtém a direção do movimento a partir do botão
    if np.array_equal(estado_atual, ESTADO_FINAL):
        return  # retorna se o jogo já estiver resolvido

    novo_estado = mover(estado_atual, direcao_movimento)  # move para o novo estado na direção especificada
    if novo_estado is not None:
        estado_atual = novo_estado  # atualiza o estado atual com o novo estado
        exibir_tabuleiro(estado_atual)  # atualiza a interface para exibir o novo estado
        verificar_objetivo()  # verifica se o jogo foi resolvido após o movimento
    else:
        print("Movimento inválido! Tente novamente.")

def verificar_objetivo():
    """Verifica se o estado atual é o estado objetivo."""
    global estado_atual  # usa a variável global estado_atual
    if np.array_equal(estado_atual, ESTADO_FINAL):  # verifica se o estado atual é igual ao estado final
        resultado.value = "Parabéns! Você resolveu o quebra-cabeça!"
        botao_resolver.disabled = True
        botao_novo_jogo.disabled = False
        botao_acima.disabled = True
        botao_abaixo.disabled = True
        botao_esquerda.disabled = True
        botao_direita.disabled = True
    else:
        resultado.value = ""  # limpa a mensagem de resultado se o jogo não estiver resolvido

def on_click_resolver(botao):
    """Callback para o botão de resolver."""
    global estado_atual  # usa a variável global estado_atual
    metodo = metodo_widget.value  # obtém o método selecionado
    resolver_quebra_cabeca(estado_atual, metodo)  # chama a função para resolver o quebra cabeça com o método escolhido

def resolver_quebra_cabeca(estado_inicial, metodo):
    """Resolve o quebra-cabeça usando o método especificado."""
    global estado_atual  # usa a variável global estado_atual
    estado_atual = estado_inicial.copy()  # copia o estado inicial para evitar alterações indesejadas
    if metodo == 'BFS':
        solucao = bfs(estado_inicial)  # resolve usando busca em largura
    else:
        solucao = a_star(estado_inicial)  # resolve usando A*

    if solucao is None:
        resultado.value = "Nenhuma solução encontrada."
        return

    resultado.value = "Solução encontrada! Movimentos: " + ', '.join(solucao)

    for direcao_movimento in solucao:
        estado_atual = mover(estado_atual, direcao_movimento)  # Move para cada direção na solução
        exibir_tabuleiro(estado_atual)  # Atualiza a interface para exibir o novo estado
        time.sleep(1)  # Pausa temporizada de 1 segundo entre os movimentos

    botao_resolver.disabled = True  # Desabilita o botão de resolver após resolver o quebra-cabeça
    botao_novo_jogo.disabled = False  # Habilita o botão de novo jogo após resolver o quebra-cabeça
    botao_acima.disabled = True  # Desabilita os botões de movimento
    botao_abaixo.disabled = True
    botao_esquerda.disabled = True
    botao_direita.disabled = True

def novo_jogo(botao):
    """Callback para o botão de novo jogo."""
    global estado_atual, resultado  # Usa as variáveis globais estado_atual e resultado
    clear_output()  # Limpa a saída atual
    estado_inicial = np.array(eval(estado_inicial_widget.value))  # Obtém o novo estado inicial do widget
    estado_atual = estado_inicial.copy()  # Copia o novo estado inicial para evitar alterações indesejadas
    resultado.value = ""  # Limpa a mensagem de resultado
    botao_resolver.disabled = False  # Habilita o botão de resolver
    botao_acima.disabled = False  # Habilita os botões de movimento
    botao_abaixo.disabled = False
    botao_esquerda.disabled = False
    botao_direita.disabled = False
    display(layout_inicial)  # Exibe o layout inicial com o novo jogo

def iniciar_jogo(botao):
    """Callback para o botão de iniciar."""
    clear_output()
    display(caixa_entrada, VBox([HBox(botoes[i]) for i in range(3)]), caixa_controle)
    exibir_tabuleiro(np.array(eval(estado_inicial_widget.value)))

    botao_iniciar.on_click(iniciar_jogo)

# Widgets da interface
botoes = [[widgets.Button(description='', layout=widgets.Layout(width='50px', height='50px')) for _ in range(3)] for _ in range(3)]
for i in range(3):
    for j in range(3):
        botoes[i][j].style.button_color = 'blue'
        botoes[i][j].style.font_color = 'white'

# botões de movimento
botao_acima = widgets.Button(description='Acima')
botao_abaixo = widgets.Button(description='Abaixo')
botao_esquerda = widgets.Button(description='Esquerda')
botao_direita = widgets.Button(description='Direita')

# configurações de eventos para os botões de movimento
botao_acima.on_click(on_click_movimento)
botao_abaixo.on_click(on_click_movimento)
botao_esquerda.on_click(on_click_movimento)
botao_direita.on_click(on_click_movimento)

# widget para inserção do estado inicial e método de resolução
estado_inicial_widget = widgets.Text(value="[[2, 0, 3], [1, 7, 4], [6, 8, 5]]", description="Estado Inicial:")
metodo_widget = widgets.Dropdown(options=['BFS', 'A*'], description="Método:")

# botões de resolver e novo jogo, e label para exibir resultado
botao_resolver = widgets.Button(description="Resolver")
botao_novo_jogo = widgets.Button(description="Novo Jogo", disabled=True)
resultado = widgets.Label(value="")

# configurações de eventos para os botões de resolver e novo jogo
botao_resolver.on_click(on_click_resolver)
botao_novo_jogo.on_click(novo_jogo)

# layout dos botões de movimento
caixa_botoes_movimento = HBox([botao_acima, botao_abaixo])
caixa_botoes_direcao = HBox([botao_esquerda, botao_direita])
caixa_controle = VBox([caixa_botoes_movimento, caixa_botoes_direcao])

# layout da entrada de dados e resultado
caixa_entrada = VBox([metodo_widget, botao_resolver, botao_novo_jogo, resultado])

# botão de iniciar
botao_iniciar = widgets.Button(description="Iniciar")

# configuração de evento para o botão de iniciar
botao_iniciar.on_click(iniciar_jogo)

# layout inicial com o botão de iniciar e entrada do estado inicial
layout_inicial = VBox([estado_inicial_widget, botao_iniciar])

# exibe apenas o layout inicial no início
display(layout_inicial)

# estado inicial do quebra-cabeça
estado_atual = np.array(eval(estado_inicial_widget.value))
